### American option pricing via simulation

In [1]:
import numpy as np
from time import time
import matplotlib.pyplot as plt
# import our polynomial function
from polynomials import constructX
# linear regression
from sklearn.linear_model import LinearRegression

Parameters

In [2]:
r = 0.10
q = 0.01
s0 = 100
K = 100
sig = 0.40
T = 1

m = 365 # daily excercise
numPaths = 10000

dt = T/m
indicators = np.zeros(numPaths)

Algorithm

In [4]:
sMin = 10
sMax = 400

typeOfPolynomial = 'Laguerre'

st = time()

#Initiate paths (not efficient; we should utilize the Brownian Bridge)
# simulate stock prices
s = np.zeros((m+1, numPaths))
t = np.linspace(0, T, m+1)
z = np.random.randn(m, numPaths)
delta_log_s = (r - q - sig*sig/2)*dt + sig*np.sqrt(dt)*z
log_s = np.zeros((m+1, numPaths))
log_s[0,:] = np.log(s0)
log_s[1:,:] = delta_log_s
log_s = np.cumsum(log_s, 0)
s = np.exp(log_s)

P_1 = np.maximum(K - s[m,:], 0)
P_2 = np.maximum(K - s[m,:], 0)
P_3 = np.maximum(K - s[m,:], 0)
vHat_1 = np.zeros(numPaths)
vHat_2 = np.zeros(numPaths)
vHat_3 = np.zeros(numPaths)

for i in np.arange(m-1, 0, -1):
    s_i = s[i, :]
    g = np.maximum(K - s_i, 0)
    # in-the-money indicator
    indicator = (g > 0)
    xi = s_i[indicator]
    xXi_1 = constructX(xi, 'Laguerre')
    xXi_2 = constructX(2*(xi - sMin)/(sMax - sMin) - 1, 'Chebychev_firstKind')
    xXi_3 = constructX(2*(xi - sMin)/(sMax - sMin) - 1, 'Chebychev_secondKind')
    yi_1 = np.exp(-r*dt)*P_1[indicator]
    yi_2 = np.exp(-r*dt)*P_2[indicator]
    yi_3 = np.exp(-r*dt)*P_3[indicator]
    # regression
    reg_1 = LinearRegression(fit_intercept=False).fit(xXi_1, yi_1)
    alpha_1 = reg_1.coef_
    reg_2 = LinearRegression(fit_intercept=False).fit(xXi_2, yi_2)
    alpha_2 = reg_2.coef_
    reg_3 = LinearRegression(fit_intercept=False).fit(xXi_3, yi_3)
    alpha_3 = reg_3.coef_
    vH_1 = np.dot(xXi_1, alpha_1)
    vH_2 = np.dot(xXi_2, alpha_2)
    vH_3 = np.dot(xXi_3, alpha_3)
    vHat_1[indicator] = vH_1
    vHat_2[indicator] = vH_2
    vHat_3[indicator] = vH_3
    
    indx_1 = np.logical_and(indicator, g > vHat_1)
    P_1[indx_1] = g[indx_1]
    P_1[~indx_1] = np.exp(-r*dt)*P_1[~indx_1]
    indx_2 = np.logical_and(indicator, g > vHat_2)
    P_2[indx_2] = g[indx_2]
    P_2[~indx_2] = np.exp(-r*dt)*P_2[~indx_2]
    indx_3 = np.logical_and(indicator, g > vHat_3)
    P_3[indx_3] = g[indx_3]
    P_3[~indx_3] = np.exp(-r*dt)*P_3[~indx_3]

et = time()
    
print('Number of sims: %i' % numPaths)
print('Elapsed times was %f seconds' % (et-st))
premiumHat_1 = np.exp(-r*dt) * np.mean(P_1)
premiumHat_2 = np.exp(-r*dt) * np.mean(P_2)
premiumHat_3 = np.exp(-r*dt) * np.mean(P_3)
print('American price (Laguerre): %f' % premiumHat_1)
print('American price (Chebychev 1st): %f' % premiumHat_2)
print('American price (Chebuchev 2nd): %f' % premiumHat_3)

Number of sims: 10000
Elapsed times was 2.535405 seconds
American price (Laguerre): 12.487710
American price (Chebychev 1st): 12.518897
American price (Chebuchev 2nd): 12.518897
